# Open synthetic data

In [2]:
import os
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

path = '/home/INT/grimaldi.a/Documents/projets/WP3/data'
list = os.listdir(path)
print(f'list of all files : \n {list}')

if not os.path.exists('../Records/'):
    os.mkdir('../Records/')
    os.mkdir('../Records/model/')
    os.mkdir('../Records/train/')
    os.mkdir('../Records/test/')

labelz = [0,0,1,2,2,2,1] # 0 for 'sea' and 1 for 'ground'
x_index, y_index, t_index, p_index = 0, 1, 2, 3

patch_R = 20
duration = None
min_events = 20
#duration = int(1e7)

def load_data_patches(path, rec_number, patch_size, time_limit, min_events):
    events_stacked = np.array([])
    label_stacked = []
    indices = [0]
    for num in rec_number:
        fname_csv = f'{path}/{list[num]}'
        df = pd.read_csv(fname_csv)
        events = np.zeros([df.shape[0],4]).astype(int)
        events[:,[x_index, y_index, p_index]] = df.values[:,[x_index,y_index,4]]
        t_sec = df.values[:,2]
        t_nsec = df.values[:,3]
        initial_time = t_sec[0]*1e6+t_nsec[0]*1e-3
        events[:,t_index] = t_sec[:]*1e6+t_nsec[:]*1e-3-initial_time
        label = labelz[num]
        print(f'file name: {list[num]} \n total number of events: {len(events)} \n recording time: {events[-1,t_index]*1e-6} sec \n \n')
        width, height = max(events[:,x_index]), max(events[:,y_index])
        pbar = tqdm(total=width//patch_size*height//patch_size)
        for x in range(width//patch_size):
            for y in range(height//patch_size):
                events_patch = events[
                               (events[:,x_index]>=x*patch_size)&(events[:,x_index]<(x+1)*patch_size)&
                               (events[:,y_index]>=y*patch_size)&(events[:,y_index]<(y+1)*patch_size)]
                events_patch[:,x_index] -= x*patch_size
                events_patch[:,y_index] -= y*patch_size
#                if time_limit:
#                    time = 0
#                    events_patch[:,t_index] = events_patch[(events_patch[:,t_index]<time)]
                if len(events_patch)<min_events:
                    pass
                else:
                    events_patch[:,t_index] -= np.min(events_patch[:,t_index])
                    indices.append(indices[-1]+events_patch.shape[0])
                    label_stacked.append(label)
                    events_stacked = np.vstack([events_stacked, events_patch]) if events_stacked.size else events_patch
                pbar.update(1)
        print(np.max(events_patch[:,x_index]), np.max(events_patch[:,y_index]), np.max(events_patch[:,t_index]))
        pbar.close()
    return events_stacked, label_stacked, indices

list of all files : 
 ['only_sea_session_2.csv', 'only_sea_session_1.csv', 'only_ground_session_2.csv', 'mixed_session_1.csv', 'mixed_session_3.csv', 'mixed_session_2.csv', 'only_ground_session_1.csv']


In [3]:
with open('../Data/aprovisynt_patches_train.pkl', 'rb') as file:
    events_train, label_train, indices_train = pickle.load(file)

# Build HOTS network

You have to install the HOTS module available at: 
https://github.com/SpikeAI/HOTS
and also this version of tonic (for the moment): 
https://github.com/SpikeAI/tonic

In [4]:
from HOTS.Network import network
from tqdm import tqdm
import pickle

#parameters
tau = 10 #ms
homeo = True
date = '2021-12-09'

aprovhots = network(timestr = date, tau = tau, homeo=homeo)
aprovhots.sensformat([patch_R,patch_R])

f_name = f'../Records/model/{aprovhots.get_fname()}.pkl'

In [5]:
nbevents = int(1e6)
pbar = tqdm(total=nbevents)
for i in range(nbevents):
    x, y, t, p = events_train[i,[x_index, y_index, t_index, p_index ]]
    aprovhots.run(x, y, t, p, learn=True, to_record=True)
    pbar.update(1)
for l in range(len(aprovhots.L)):
    aprovhots.stats[l].histo = aprovhots.L[l].cumhisto.copy()
pbar.close()
aprovhots.plotlayer();

  2%|███▊                                                                                                                                                        | 24643/1000000 [00:04<02:07, 7627.15it/s]/home/INT/grimaldi.a/Documents/projets/HOTS/HOTS/HOTS/TimeSurface.py:84: RuntimeWarning: overflow encountered in exp
  self.spatpmat = self.spatpmat*np.exp(-(float(tev-self.t))/self.tau)
/home/INT/grimaldi.a/Documents/projets/HOTS/HOTS/HOTS/TimeSurface.py:84: RuntimeWarning: invalid value encountered in multiply
  self.spatpmat = self.spatpmat*np.exp(-(float(tev-self.t))/self.tau)
 16%|████████████████████████▎                                                                                                                                  | 156655/1000000 [01:37<06:29, 2165.82it/s]

KeyboardInterrupt: 

In [7]:
with open(f_name, 'wb') as file:
    pickle.dump(aprovhots, file, pickle.HIGHEST_PROTOCOL)

# MLR on a set of events
## Transform event stream with HOTS

In [2]:
with open(f_name, 'rb') as file:
    aprovhots = pickle.load(file)

In [8]:
timout = []
xout = []
yout = []
polout = []

nbevents = min(len(events_train))
pbar = tqdm(total=nbevents)
inc = 0
for i in range(nbevents):
    x, y, t, p = events_train[i,[x_index, y_index, t_index, p_index ]]
    out, activout = aprovhots.run(x, y, t, p, learn=False, to_record=False)
    if activout:
        xout.append(out[0])
        yout.append(out[1])
        timout.append(out[2])
        polout.append(out[3])
        pbar.update(1)
    if indices_train[inc]==i:
        print(label_train[inc])
        inc+=1
camsize = aprovhots.TS[-1].camsize
nbpola = aprovhots.L[-1].kernel.shape[1]
eventsout = [xout,yout,timout,polout]
pbar.close()

f_name = f'../Records/train/{aprovhots.get_fname()}.pkl'
with open(f_name, 'wb') as file:
    pickle.dump(eventsout, file, pickle.HIGHEST_PROTOCOL)

TypeError: 'int' object is not iterable

## Build a MLR model with outputs of HOTS